# Descarga de datos

## URL de las distintas fuentes de datos

**PRODUCCIÓN ANUAL AGRÍCOLA**
- https://nube.siap.gob.mx/cierreagricola/

**PRODUCCIÓN MENSUAL AGRÍCOLA**
- https://nube.siap.gob.mx/avance_agricola/
- http://infosiap.siap.gob.mx:8080/agricola_siap_gobmx/ResumenProducto.do

**INTENCION DE SIEMBRA**
- https://nube.siap.gob.mx/agroprograma/

**SISTEMA DE INFORMACIÓN AGROALIMENTARIA DE CONSULTA**
- http://infosiap.siap.gob.mx/gobmx/datosAbiertos_a.php

**ESTADÍSTICA DE USO TECNOLÓGICO Y DE SERVICIOS EN LA SUPERFICIE AGRÍCOLA**
- Fertilizada: https://nube.siap.gob.mx/index.php/s/kTLGLipCBEBiEFM/download
- Semilla mejorada y semilla criolla: https://nube.siap.gob.mx/index.php/s/OjKcZZL8QClaK6U/download
- Mecanizada: https://nube.siap.gob.mx/index.php/s/P7yK8NWmppStv7i/download
- Riego: https://nube.siap.gob.mx/index.php/s/cz6nETNW4b0krzD/download
- Sanidad: https://nube.siap.gob.mx/index.php/s/0l0Vw6VUf8fQ0dI/download
- Asistencia técnica: https://nube.siap.gob.mx/index.php/s/EHaUuYSEnHqHLqA/download

**INFORMACIÓN AGRÍCOLA DE GÉNERO**
- 2015 - 2017: https://www.gob.mx/cms/uploads/attachment/file/399835/Estadistica_de_G_nero_2017.pdf
- 2016 - 2018: https://www.gob.mx/cms/uploads/attachment/file/515428/Participaci_n_de_mujeres_en_valor_2016_17_18.pdf
- 2017 - 2019: https://www.gob.mx/cms/uploads/attachment/file/630415/Estadistica_de_Genero_2019.pdf

**MONOGRAFÍAS DE CULTIVOS**
- https://www.gob.mx/siap/documentos/monografias



## Librerias

In [ ]:
# !pip install lxml
# !pip install pandas
# !pip install beautifulsoup4
# !pip install pyarrow

In [51]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import requests


## Catalogos para API: JSON de Victor

Nota: sí vienen los 64 cultivos que aparecen en la página

In [52]:
if not os.path.exists('catalogos_api'):
    os.mkdir("catalogos_api")

df = pd.read_json('Victor-dict.json',orient='index')

tag = ["municipio","anioagric","cicloProd","modalidad","entidad","cultivo","distrito","mesagric"]
name_csv = ['Municipio','Anio_Agricola','Ciclo_Prod','Modalidad','Entidad','Cultivo','Distrito',"Mes_Agricola"]

for i in range(8):
    # Toma los datos pertenecientes al tag
    df_aux2 = df.loc[df.index.str.startswith(tag[i])]
    df_aux = df_aux2.copy()
    # Elimina la str del tag+_ de la columna index
    df_aux.index = df_aux.index.str.replace(tag[i]+'_', "")
    # Convierte el indice en columna
    df_aux.reset_index(inplace=True)
    # Cambia los nombres de las columnas
    df_aux.rename(columns={0: "ID","index":"Name"}, inplace=True)
    # Crea el path donde se guardara
    ruta_completa = os.path.join("catalogos_api",name_csv[i])
    # Guardar el DataFrame en el archivo CSV en la carpeta "catalogos_api"
    df_aux.to_csv(ruta_completa + '.csv', index=False)

#Elimina los df
del df_aux, df_aux2

## Descarga de Datos Raw

In [55]:
# Creacion del folder raw
if not os.path.exists('raw'):
    os.mkdir("raw")

In [56]:
# Comando para ignorar warnings (salen en cada request)
warnings.filterwarnings('ignore')

In [142]:
anio_arr = pd.read_csv('./catalogos_api/Anio_Agricola.csv').ID.to_list()
cultivo_arr = pd.read_csv('./catalogos_api/Cultivo.csv').ID.to_list()
mes_arr = pd.read_csv('./catalogos_api/Mes_Agricola.csv').ID.to_list()

In [143]:
cultivo_arr.remove(0) # 0: Todos

In [145]:
for i in anio_arr:
    if not os.path.exists('raw/'+str(i)):
        os.mkdir("raw/"+str(i))

In [146]:
def API_SIAP(anio,mes,cultivo):
  # Endpoint
  url = "https://nube.siap.gob.mx/avance_agricola/"

  payload = {'xajax': 'reporte', # Para obtener la tabla
  'xajaxr': '1696449941927', # Timestamp UNIX
  'xajaxargs[]': [
      '1', # 1: Desglose por estados, 2: Cultivo total
      str(anio), # Anio
      '5', # ID Ciclo
      '3', # ID Modalidad
      '0', # ID Estado (0: Nacional)
      '--',
      '--',
      str(cultivo), # ID Cultivo
      '200201',
      '0',
      '1', # 1: Por municipio
      'undefined',
      'undefined',
      'undefined',
      str(mes) # Valor del mes: va de 1 a 8, con 1 siendo Enero y 8 Agosto
      ]
  }

  headers = {
    'Cookie': 'PHPSESSID=45ri2k73cbp2iptcrufu88p360'
  }

  # Hacemos el request
  response = requests.request("POST", url, headers=headers, data=payload, verify=False)
  response.encoding='ISO-8859-1'

  return response.text

In [149]:
for anio in anio_arr:
    for mes in mes_arr:
        columnas = ["Entidad", "Municipio", "Superficie Sembrada", "Superficie Cosechada", "Superficie Siniestrada", "Produccion", "Rendimiento","Anio", "Mes", "Cultivo"]
        df_raw = pd.DataFrame(columns=columnas)
        for cultivo in cultivo_arr:
            xml_data = API_SIAP(anio,mes,cultivo)
            # Parseamos el XML con BeautifulSoup usando el analizador XML (lxml)
            soup = BeautifulSoup(xml_data, 'lxml')

            # Encontramos la tabla por su ID
            tabla = soup.find('table', id='Resultados-reporte')

            if tabla:
                # Inicializamos listas para almacenar los datos
                datos = []

                # Iteramos a través de las filas de la tabla
                for fila in tabla.find_all('tr')[1:]:  # Ignoramos la primera fila de encabezados
                    columnas = fila.find_all('td')
                    if len(columnas) == 0:  # Ignoramos las filas vacías
                        continue

                    # Extraemos los datos relevantes de cada columna
                    #posicion = columnas[0].get_text()
                    entidad = columnas[1].get_text()
                    municipio = columnas[2].get_text()
                    superficie_sembrada = columnas[3].get_text()
                    superficie_cosechada = columnas[4].get_text()
                    superficie_siniestrada = columnas[5].get_text()
                    produccion = columnas[6].get_text()
                    rendimiento = columnas[7].get_text()

                    # Agregamos los datos a la lista
                    datos = [
                        entidad, municipio,
                        superficie_sembrada, superficie_cosechada,
                        superficie_siniestrada, produccion, rendimiento,
                        anio, mes, cultivo
                    ]
                    df_raw.loc[len(df_raw)] = datos
        name_csv = 'Avance_Agricola_'+str(anio)+'_'+str(mes)
        df_raw.to_csv('./raw/'+str(anio)+'/'+name_csv+'.csv', index=False)

## Preparación de datos tidy

### Unión de los datos

In [166]:
# anio,mes,ciclo,cultivo, entidad, modalidad
columnas = ["Entidad", "Municipio", "Superficie Sembrada", "Superficie Cosechada", "Superficie Siniestrada", "Produccion", "Rendimiento","Anio", "Mes", "Cultivo"]
df_raw = pd.DataFrame(columns=columnas)

total = 0
for anio in anio_arr:
    for i in range(1,13):
        df_aux = pd.read_csv("./raw/"+str(anio)+"/Avance_Agricola_"+str(anio)+ "_" + str(i) + '.csv')
        df_raw = pd.concat([df_raw, df_aux], axis=0)
        total += df_aux.shape[0]

df_raw['Superficie Siniestrada'] = pd.to_numeric(df_raw['Superficie Siniestrada'], errors='coerce')

In [167]:
if not os.path.exists('raw/Avance_Agricola_Historico.parquet'):
    name_csv = "Avance_Agricola_Historico"
    ruta_completa = os.path.join("raw",name_csv)
    df_raw.to_parquet(ruta_completa + '.parquet', index=False)

In [168]:
print("Total de datos: ",total)

Total de datos:  992213


### Decodificación de variables

In [169]:
df = pd.read_parquet('./raw/Avance_Agricola_Historico.parquet')

In [170]:
catalogo_cultivos = pd.read_csv('./catalogos_api/Cultivo.csv')
catalogo_mes = pd.read_csv('./catalogos_api/Mes_Agricola.csv')

catalogo_cultivos = catalogo_cultivos.set_index('ID')['Name'].to_dict()
catalogo_mes = catalogo_mes.set_index('ID')['Name'].to_dict()

df['Cultivo'] = df['Cultivo'].map(catalogo_cultivos)
df['Mes'] = df['Mes'].map(catalogo_mes)


In [171]:
df

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,Anio,Mes,Cultivo
0,Aguascalientes,Aguascalientes,8.00,0.00,0.0,0.00,0,2023,Enero,Agave
1,Baja California Sur,La Paz,3.00,0.00,0.0,0.00,0,2023,Enero,Agave
2,Colima,Manzanillo,86.29,0.00,0.0,0.00,0,2023,Enero,Agave
3,Durango,Durango,13.00,0.00,0.0,0.00,0,2023,Enero,Agave
4,Durango,Mezquital,110.00,0.00,0.0,0.00,0,2023,Enero,Agave
...,...,...,...,...,...,...,...,...,...,...
992208,Puebla,Domingo Arenas,12.00,12.00,0.0,110.00,9.17,2018,Diciembre,Zarzamora
992209,Puebla,Huejotzingo,10.00,10.00,0.0,98.00,9.80,2018,Diciembre,Zarzamora
992210,Puebla,Tochimilco,1.00,1.00,0.0,4.10,4.10,2018,Diciembre,Zarzamora
992211,Querétaro,San Juan del Río,3.00,3.00,0.0,23.10,7.70,2018,Diciembre,Zarzamora


In [172]:
df.to_parquet('tidy.parquet', index=False)

In [173]:
df = pd.read_parquet('tidy.parquet')
df

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,Anio,Mes,Cultivo
0,Aguascalientes,Aguascalientes,8.00,0.00,0.0,0.00,0,2023,Enero,Agave
1,Baja California Sur,La Paz,3.00,0.00,0.0,0.00,0,2023,Enero,Agave
2,Colima,Manzanillo,86.29,0.00,0.0,0.00,0,2023,Enero,Agave
3,Durango,Durango,13.00,0.00,0.0,0.00,0,2023,Enero,Agave
4,Durango,Mezquital,110.00,0.00,0.0,0.00,0,2023,Enero,Agave
...,...,...,...,...,...,...,...,...,...,...
992208,Puebla,Domingo Arenas,12.00,12.00,0.0,110.00,9.17,2018,Diciembre,Zarzamora
992209,Puebla,Huejotzingo,10.00,10.00,0.0,98.00,9.80,2018,Diciembre,Zarzamora
992210,Puebla,Tochimilco,1.00,1.00,0.0,4.10,4.10,2018,Diciembre,Zarzamora
992211,Querétaro,San Juan del Río,3.00,3.00,0.0,23.10,7.70,2018,Diciembre,Zarzamora
